finally

In [14]:
import numpy as np
import pandas as pd
import cv2
from ultralytics import YOLO
model = YOLO('C:/salman/sir_pro/runs/detect/train/weights/best.pt')


class LaneDetectionAPI:



    def __init__(self, video_path=0):
        self.video_path = video_path
        self.cap = None




    def canny_detector(self,image):

        
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Create masks for white and yellow color    
        lower_white = np.array([0, 0, 200], dtype=np.uint8)
        upper_white = np.array([180, 30, 255], dtype=np.uint8)
        lower_yellow = np.array([20, 100, 100], dtype=np.uint8)
        upper_yellow = np.array([40, 255, 255], dtype=np.uint8)
        
        white_mask = cv2.inRange(hsv, lower_white, upper_white)
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Combine the masks to get the final mask
        combined_mask = cv2.bitwise_or(white_mask, yellow_mask)
        
        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(image, image, mask=combined_mask)
        
        # Convert the masked image to grayscale
        grayscale = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)


        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(grayscale, (9,9), 0)

        # Apply Canny edge detection
        edges = cv2.Canny(blurred, 100, 200)

        return edges


    def region_selection(self,image):
        """
        Determine and cut the region of interest in the input image.
            Parameters:
                image: An np.array compatible with plt.imshow.
        """
        mask = np.zeros_like(image)   
        #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(image.shape) > 2:
            channel_count = image.shape[2]
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        #We could have used fixed numbers as the vertices of the polygon,
        #but they will not be applicable to images with different dimesnions.
        rows, cols = image.shape[:2]
        bottom_left  = [cols * 0.1, rows * 0.95]
        top_left     = [cols * 0.4, rows * 0.62]
        bottom_right = [cols * 0.9, rows * 0.95]
        top_right    = [cols * 0.6, rows * 0.62]
        vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(image, mask)
        return masked_image


    def hough_transform(self,image):
        """
        Determine and cut the region of interest in the input image.
            Parameters:
                image: The output of a Canny transform.
        """
        rho = 1              #Distance resolution of the accumulator in pixels.
        theta = np.pi/180    #Angle resolution of the accumulator in radians.
        threshold = 20       #Only lines that are greater than threshold will be returned.
        minLineLength = 20   #Line segments shorter than that are rejected.
        maxLineGap = 300     #Maximum allowed gap between points on the same line to link them
        return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
                            minLineLength = minLineLength, maxLineGap = maxLineGap)



    def draw_lines(self,image, lines, color = [255, 0, 0], thickness = 3):
        """
        Draw lines onto the input image.
            Parameters:
                image: An np.array compatible with plt.imshow.
                lines: The lines we want to draw.
                color (Default = red): Line color.
                thickness (Default = 2): Line thickness.
        """
        image = np.copy(image)
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(image, (x1, y1), (x2, y2), color, thickness)
        return image



    def average_slope_intercept(self,lines):
        """
        Find the slope and intercept of the left and right lanes of each image.
            Parameters:
                lines: The output lines from Hough Transform.
        """
        if lines is None:
            return None, None

        left_lines    = [] #(slope, intercept)
        left_weights  = [] #(length,)
        right_lines   = [] #(slope, intercept)
        right_weights = [] #(length,)
        
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x1 == x2:
                    continue
                slope = (y2 - y1) / (x2 - x1)
                intercept = y1 - (slope * x1)
                length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
                if slope < 0:
                    left_lines.append((slope, intercept))
                    left_weights.append((length))
                else:
                    right_lines.append((slope, intercept))
                    right_weights.append((length))
        left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
        right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
        return left_lane, right_lane


    def pixel_points(self,y1, y2, line):
        """
        Converts the slope and intercept of each line into pixel points.
            Parameters:
                y1: y-value of the line's starting point.
                y2: y-value of the line's end point.
                line: The slope and intercept of the line.
        """
        if line is None:
            return None

        slope, intercept = line
        
        # Check if the slope is not equal to zero (not vertical)
        if slope != 0:
            x1 = int((y1 - intercept) / slope)
            x2 = int((y2 - intercept) / slope)
            y1 = int(y1)
            y2 = int(y2)
            return ((x1, y1), (x2, y2))
        else:
            return None


    def lane_lines(self,image, lines):
        """
        Create full lenght lines from pixel points.
            Parameters:
                image: The input test image.
                lines: The output lines from Hough Transform.
        """
        left_lane, right_lane = self.average_slope_intercept(lines)
        y1 = image.shape[0]
        y2 = y1 * 0.6
        left_line  = self.pixel_points(y1, y2, left_lane)
        right_line = self.pixel_points(y1, y2, right_lane)
        return left_line, right_line

        
    def draw_lane_lines(self,image, lines, color=[0, 255, 0], thickness=3):
        """
        Draw lines onto the input image.
            Parameters:
                image: The input test image.
                lines: The output lines from Hough Transform.
                color (Default = red): Line color.
                thickness (Default = 12): Line thickness. 
        """
        line_image = np.zeros_like(image)
        for line in lines:
            if line is not None:
                cv2.line(line_image, *line,  color, thickness)
        return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)



    def frame_processor(self,image):
        """
        Process the input frame to detect lane lines.
            Parameters:
                image: Single video frame.
        """
        
        edges        = self.canny_detector(image)
        region       = self.region_selection(edges)
        hough        = self.hough_transform(region)
        result       = self.draw_lane_lines(image, self.lane_lines(image, hough))
        return result 



    def process_video(self):

        cap = cv2.VideoCapture(self.video_path)



        while (cap.isOpened()): 
            ret,frame=cap.read()
            if ret==True:
                combo_frame = self.frame_processor(frame)
                results =model.predict(source=combo_frame,show=True)
            
                if cv2.waitKey(1) & 0xFF == ord('q'):   #press to exit
                    break
            else:
                break

        cap.release()
        cv2.destroyAllWindows()



In [19]:
api = LaneDetectionAPI("ved.mp4")
api.process_video()

0: 384x640 (no detections), 238.4ms
Speed: 3.0ms preprocess, 238.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 416.8ms
Speed: 23.0ms preprocess, 416.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 189.5ms
Speed: 4.8ms preprocess, 189.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 214.4ms
Speed: 4.0ms preprocess, 214.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 177.5ms
Speed: 3.0ms preprocess, 177.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 190.5ms
Speed: 1.9ms preprocess, 190.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 178.5ms
Speed: 4.9ms preprocess, 178.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 203.5ms
Speed: 2.9ms prepr